In [ ]:
---
comments: true
layout: post
title: CSSE 2 Final Week 2
author: Lucas Masterson
description: Personal blog for CSSE 2 Final Week 22
permalink: /csse/final/week22
type: ccc
---

## Day 1

Decided to add some options (one) to damage the boss. Here, I give to ability to collect "shards" (mind the shamelessly copied `Coin.js` code):

```js
// Shard.js
import BossFight from './BossFight.js';
import GameControl from './GameControl.js';
import GameEnv from './GameEnv.js';
import GameObject from './GameObject.js';

export class Shard extends GameObject {
    constructor(canvas, image, data) {
        super(canvas, image, data, 0.5, 0.5);
        this.coinX = Math.random() * GameEnv.innerWidth;
        this.coinY = 0.5 + Math.random() * 0.5; // Limit to bottom half of screen
        this.size();
        this.id = this.initiateId();
    }

    initiateId() {
        const currentCoins = GameEnv.gameObjects

        return currentCoins.length //assign id to the coin's position in the gameObject Array (is unique to the coin)
    }

    // Required, but no update action
    update() {
        this.collisionChecks()
    }

    // Draw position is always 0,0
    draw() {
        // Save the current transformation matrix
        this.ctx.save();

        // Rotate the canvas 90 degrees to the left
        this.ctx.rotate(-Math.PI / 2);

        // Draw the image at the rotated position (swap x and y)
        this.ctx.drawImage(this.image, -this.image.height, 0);

        // Restore the original transformation matrix
        this.ctx.restore();
    }

    // Center and set Coin position with adjustable height and width
    size() {
        if (this.id) {
            if (GameEnv.claimedCoinIds.includes(this.id)) {
                this.hide()
            }
        }

        const scaledWidth = this.image.width * 0.2;
        const scaledHeight = this.image.height * 0.169;

        const coinX = this.coinX;
        const coinY = (GameEnv.bottom - scaledHeight) * this.coinY;

        // Set variables used in Display and Collision algorithms
        this.bottom = coinY;
        this.collisionHeight = scaledHeight;
        this.collisionWidth = scaledWidth;

        this.canvas.style.width = `${scaledWidth}px`;
        this.canvas.style.height = `${scaledHeight}px`;
        this.canvas.style.position = 'absolute';
        this.canvas.style.left = `${coinX}px`;
        this.canvas.style.top = `${coinY}px`;
    }
    collisionAction() {
        // check player collision
        if (this.collisionData.touchPoints.other.id === "player") {
            if (this.id) {
            GameEnv.claimedCoinIds.push(this.id);
            }
            this.destroy();

            GameControl.gainCoin(5);
            GameEnv.playSound("coin");
            
            console.log("Shard colleted");
            const boss = GameEnv.gameObjects.find(obj => obj instanceof BossFight);
            if (boss) {
                boss.currentHp -= 50;
                console.log("Boss damaged by shard");
            }
        }
    }
    
    // Method to hide the coin
    hide() {
        this.canvas.style.display = 'none';
    }

    // Method to show the coin
    show() {
        this.canvas.style.display = 'block';
    }

    reset() {
        this.coinX =Math.random() * GameEnv.innerWidth;
        this.coinY = 0.5 + Math.random() * 0.5;
        this.size();
        this.show();
    }
}

export default Shard;
```

And you'll probably notice that this is shamelessly recycled Coin code with added randomness and boss damage. Why, yes it is! Good eye!

Anyhow, here two other changes I made:

Loading the coins into the level

```js
// GameSetterBossFight.js
    { name: 'shard', id: 'coin', class: Shard, data: assets.obstacles.coin, xPercentage: 0.5, yPercentage: 0.5 },
    { name: 'shard', id: 'coin', class: Shard, data: assets.obstacles.coin, xPercentage: 0.5, yPercentage: 0.5 },
    { name: 'shard', id: 'coin', class: Shard, data: assets.obstacles.coin, xPercentage: 0.5, yPercentage: 0.5 },
    { name: 'shard', id: 'coin', class: Shard, data: assets.obstacles.coin, xPercentage: 0.5, yPercentage: 0.5 },
    { name: 'shard', id: 'coin', class: Shard, data: assets.obstacles.coin, xPercentage: 0.5, yPercentage: 0.5 },
```

and also

```js
// BossFight.js
// Method to handle player death (recycled from SkibidiTitan.js)
kill(target) {
    target.canvas.style.transition = "transform 0.5s";
    target.canvas.style.transform = "rotate(-90deg) translate(-26px, 0%)";
    GameEnv.playSound("PlayerDeath");

    if (target.state.isDying === false) {
        target.state.isDying = true;
        setTimeout(async () => {
            await GameControl.transitionToLevel(GameEnv.levels[GameEnv.levels.indexOf(GameEnv.currentLevel)]);
            console.log("level restart");

            GameEnv.gameObjects.forEach(obj => {
                if (obj instanceof Shard) {
                    obj.reset();
                }
            });
            target.state.isDying = false;
        }, 900);
    }
}
```

What I had to do was connect `Shard.js` and `BossFight.js` (easy enough, just import one into the other), then invoke one of the methods (for hp loss in this case). And done! You can damage the boss.

Next step is to fix the hp bar still appearing even after the boss is killed, but this was someone else's code I can't really be bothered to look at it right now -- preoccupied with Rust :3